In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle datasets download -d felipekitamura/head-ct-hemorrhage

head-ct-hemorrhage.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip "/content/head-ct-hemorrhage.zip" -d "/content"

Archive:  /content/head-ct-hemorrhage.zip
  inflating: /content/head_ct/head_ct/000.png  
  inflating: /content/head_ct/head_ct/001.png  
  inflating: /content/head_ct/head_ct/002.png  
  inflating: /content/head_ct/head_ct/003.png  
  inflating: /content/head_ct/head_ct/004.png  
  inflating: /content/head_ct/head_ct/005.png  
  inflating: /content/head_ct/head_ct/006.png  
  inflating: /content/head_ct/head_ct/007.png  
  inflating: /content/head_ct/head_ct/008.png  
  inflating: /content/head_ct/head_ct/009.png  
  inflating: /content/head_ct/head_ct/010.png  
  inflating: /content/head_ct/head_ct/011.png  
  inflating: /content/head_ct/head_ct/012.png  
  inflating: /content/head_ct/head_ct/013.png  
  inflating: /content/head_ct/head_ct/014.png  
  inflating: /content/head_ct/head_ct/015.png  
  inflating: /content/head_ct/head_ct/016.png  
  inflating: /content/head_ct/head_ct/017.png  
  inflating: /content/head_ct/head_ct/018.png  
  inflating: /content/head_ct/head_ct/019.png 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import glob

In [5]:
labels_df = pd.read_csv('labels.csv')
labels = np.array(labels_df[' hemorrhage'].tolist())

files = sorted(glob.glob('head_ct/head_ct/*.png'))
images = np.array([cv2.imread(path) for path in files])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [6]:
images_df = pd.DataFrame(images, columns=['image'])

In [7]:
images_df['width'] = images_df['image'].apply(lambda x: x.shape[0])
images_df['height'] = images_df['image'].apply(lambda x: x.shape[1])

In [8]:
images = np.array([cv2.resize(image, (128, 128)) for image in images])

In [10]:
indicies = np.random.permutation(100)
train_true_idx, test_true_idx = indicies[:90], indicies[90:]
train_false_idx, test_false_idx = indicies[:90] + 100, indicies[90:] + 100
train_idx, test_idx = np.append(train_true_idx, train_false_idx), np.append(test_true_idx, test_false_idx)

train_validationX, train_validationY = images[train_idx], labels[train_idx]
testX, testY = images[test_idx], labels[test_idx]

print(train_validationX.shape, testX.shape)
print(train_validationY.shape, testY.shape)

(180, 128, 128, 3) (20, 128, 128, 3)
(180,) (20,)


In [11]:
tr_len = train_validationX.shape[0]
train_val_split = int(tr_len*0.9)
indicies = np.random.permutation(tr_len)
train_idx, validation_idx = indicies[:train_val_split], indicies[train_val_split:]

trainX, trainY = train_validationX[train_idx], train_validationY[train_idx]
validationX, validationY = train_validationX[validation_idx], train_validationY[validation_idx]

print(trainX.shape, validationX.shape)
print(trainY.shape, validationY.shape)

(162, 128, 128, 3) (18, 128, 128, 3)
(162,) (18,)


In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import math

In [13]:
train_image_data = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.,
    zoom_range=0.05,
    rotation_range=180,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant',
    cval=0
)
validation_image_data = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.,
    zoom_range=0.05,
    rotation_range=90,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant',
    cval=0)

In [15]:
def check_accuracy(model, setX, actual, print_images=True):
    predicted = np.array([int(x[0] > 0.5) for x in model.predict(setX)])
    if print_images:
        rows = math.ceil(len(predicted)/10.)
        plt.figure(figsize=(20, 3 * rows))
        for i in range(len(predicted)):
            plt.subplot(rows, 10, i+1)
            plt.imshow(setX[i])
            plt.title("pred "+str(predicted[i])+" actual "+str(actual[i]))
        
    confusion = confusion_matrix(actual, predicted)
    tn, fp, fn, tp = confusion.ravel()
    print("True positive:", tp, ", True negative:", tn,
          ", False positive:", fp, ", False negative:", fn)

    print("Total accuracy:", np.sum(predicted==actual) / len(predicted) * 100., "%")
    return (tn, fp, fn, tp)

In [16]:
def imbalance_set(coeff=2):
    imbalanced_trainX = []
    imbalanced_trainY = []
    for i, train_x in enumerate(trainX):
        def add_entry(x, y):
            imbalanced_trainX.append(x)
            imbalanced_trainY.append(y)

        add_entry(train_x, trainY[i])

        if(trainY[i] == 1):
            for j in range(coeff-1):
                add_entry(train_x, trainY[i])
    return (np.array(imbalanced_trainX), np.array(imbalanced_trainY))

imbalanced_trainX, imbalanced_trainY = imbalance_set(2)
print(imbalanced_trainX.shape, imbalanced_trainY.shape)

(243, 128, 128, 3) (243,)


In [21]:
def Model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu'))
    
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.4))
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(1, activation='sigmoid'))
    return model

In [22]:
model = Model((128, 128, 3))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
global_average_pooling2d_2 ( (None, 64)                0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)               

In [23]:
model.fit_generator(train_image_data.flow(imbalanced_trainX, imbalanced_trainY, batch_size=4),
    steps_per_epoch=32,
    validation_data=validation_image_data.flow(validationX, validationY, batch_size=16),
    validation_steps=100,
    epochs=24)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/24
32/32 [==============================] - 2s 38ms/step - loss: 0.6796 - accuracy: 0.5939 - val_loss: 0.7378 - val_accuracy: 0.5000
Epoch 2/24
32/32 [==============================] - 1s 29ms/step - loss: 0.6269 - accuracy: 0.6984
Epoch 3/24
32/32 [==============================] - 1s 29ms/step - loss: 0.5923 - accuracy: 0.7335
Epoch 4/24
32/32 [==============================] - 1s 28ms/step - loss: 0.6197 - accuracy: 0.6683
Epoch 5/24
32/32 [==============================] - 1s 29ms/step - loss: 0.6574 - accuracy: 0.6452
Epoch 6/24
32/32 [==============================] - 1s 28ms/step - loss: 0.6241 - accuracy: 0.7030
Epoch 7/24
32/32 [==============================] - 1s 29ms/step - loss: 0.6516 - accuracy: 0.6976
Epoch 8/24
32/32 [==============================] - 1s 29ms/step - loss: 0.6216 - accuracy: 0.6771
Epoch 9/24
32/32 [==============================] - 1s 29ms/step - loss: 0.6601 - accuracy: 0.5658
Epoch 10/24
32/32 [==============================] - 1s 29ms/step -

In [24]:
check_accuracy(model, trainX/255., trainY, False)

True positive: 81 , True negative: 0 , False positive: 81 , False negative: 0
Total accuracy: 50.0 %


(0, 81, 0, 81)

In [25]:
check_accuracy(model, validationX/255., validationY, False)

True positive: 9 , True negative: 0 , False positive: 9 , False negative: 0
Total accuracy: 50.0 %


(0, 9, 0, 9)

In [26]:

from keras.preprocessing import image

img_width, img_height = 128, 128
img = image.load_img('/content/head_ct/head_ct/000.png', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
np.argmax(model.predict(img), axis=1)

array([0])